In [ ]:
!pip install --upgrade tensorflow-io

import os
import re
import math
import random
import warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import gridspec
import cv2
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import datasets, layers, models
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from sklearn.model_selection import KFold, train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import precision_score, recall_score, confusion_matrix

# Set TensorFlow version for display purposes
print('Tensorflow version ' + tf.__version__)

In [ ]:
pretrained_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

In [ ]:
x = pretrained_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation='relu')(x)
predictions = Dense(2, activation='softmax')(x)

In [ ]:
model1 = Model(inputs=pretrained_model.input, outputs=predictions)

for layer in pretrained_model.layers:
    layer.trainable = False

In [ ]:
x = GlobalAveragePooling2D()(pretrained_model.output)
x = Dense(256, activation='relu')(x)
predictions = Dense(2, activation='softmax')(x)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
TRAIN_PATH = '/content/drive/My Drive/DFU/Patches'
TEST_PATH = '/content/drive/My Drive/DFU/TestSet'

In [ ]:
fpath=TRAIN_PATH+r'/Abnormal(Ulcer)/10.jpg'
img=plt.imread(fpath)
print (img.shape)

(224, 224, 3)


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
    rescale=1.0/255.0,
    validation_split=0.2
)

train_ds = datagen.flow_from_directory(
    TRAIN_PATH,
    target_size=(224,224),
    batch_size=64,
    class_mode='categorical',
    subset='training',
    seed=0
)
train_labels = train_ds.classes

val_ds = datagen.flow_from_directory(
    TRAIN_PATH,
    target_size=(224,224),
    batch_size=64,
    class_mode='categorical',
    subset='validation',
    seed=0

)
val_labels = val_ds.classes

Found 507 images belonging to 2 classes.
Found 126 images belonging to 2 classes.


In [ ]:
def decode_img(img):
  img = tf.io.decode_jpeg(img, channels=3)
  return tf.image.resize(img, [224,224])

def process_path(file_path):
  img = tf.io.read_file(file_path)
  img = decode_img(img)
  return img

test_ds = tf.data.Dataset.list_files(str(TEST_PATH + '/*'), shuffle=False)
test_ds = test_ds.map(process_path, num_parallel_calls=tf.data.AUTOTUNE)

In [ ]:
class_names = train_ds.class_indices
class_names = list(class_names.keys())
print(class_names)

['Abnormal(Ulcer)', 'Normal(Healthy skin)']


In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(7, 7))
for images, labels in train_ds:
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(images[i])
        plt.title(class_names[labels[i].argmax()])
        plt.axis('off')
    break
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize = (7, 7))

i = 0
for images in test_ds.take(6):
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(images.numpy().astype("uint8"))
    plt.axis("off")
    i += 1

In [ ]:
model1.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
import tensorflow.keras.callbacks as callbacks

lr_callback = callbacks.ReduceLROnPlateau(monitor='val_accuracy', factor=0.1, patience=5)
stop_callback = callbacks.EarlyStopping(monitor='val_accuracy', patience=10)

In [ ]:
history=model1.fit(train_ds,   callbacks=[lr_callback, stop_callback],batch_size=32, epochs=30, validation_data=val_ds)

In [ ]:
plt.figure(figsize = (30, 30))
plt.tight_layout()

i = 0
for images in test_ds.take(15):
    ax = plt.subplot(5, 5, i + 1)
    prediction = np.argmax(model1.predict(np.array([images])), axis = 1)
    plt.imshow(images.numpy().astype("uint8"))
    plt.title(class_names[prediction[0]])
    plt.axis("off")

    i += 1

In [ ]:
def plot_hist(hist):
    plt.plot(hist.history["accuracy"])
    plt.plot(hist.history["val_accuracy"])
    plt.title("model accuracy")
    plt.ylabel("accuracy")
    plt.xlabel("epoch")
    plt.legend(["train", "validation"], loc="upper left")
    plt.show()


plot_hist(history)

In [ ]:
model1.save("/content/drive/My Drive/DFU/")